# Exercise 4

Match searches with bookings

### Step 0: Load libraries and data

#### Load libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
path_cleaned_bookings = "../Data/cleaned_bookings.csv.bz2" # From Exercise 2
path_cleaned_searches = "../Data/cleaned_searches.csv.bz2" # From Exercise 3

path_ext_searches = "../Data/ext_searches.csv.bz2"
path_ext_searches2 = "../Data/ext_searches2.csv.bz2"

#### Load Bookings file

In [3]:
bookings = (
    pd.read_csv(
        path_cleaned_bookings,
        compression='bz2',
        engine='c',
        delimiter="^",
        usecols=["dep_port", "arr_port", "brd_time"],
    )
)

In [4]:
bookings.shape

(1016377, 3)

#### Load Searches file

In [5]:
searches = ( 
    pd.read_csv(
        path_cleaned_searches,
        compression='bz2',
        engine='c',
        delimiter="^",
    )
)

In [6]:
searches.shape

(438240, 45)

In [7]:
searches.columns

Index(['Date', 'Time', 'TxnCode', 'OfficeID', 'Country', 'Origin',
       'Destination', 'RoundTrip', 'NbSegments', 'Seg1Departure',
       'Seg1Arrival', 'Seg1Date', 'Seg1Carrier', 'Seg1BookingCode',
       'Seg2Departure', 'Seg2Arrival', 'Seg2Date', 'Seg2Carrier',
       'Seg2BookingCode', 'Seg3Departure', 'Seg3Arrival', 'Seg3Date',
       'Seg3Carrier', 'Seg3BookingCode', 'Seg4Departure', 'Seg4Arrival',
       'Seg4Date', 'Seg4Carrier', 'Seg4BookingCode', 'Seg5Departure',
       'Seg5Arrival', 'Seg5Date', 'Seg5Carrier', 'Seg5BookingCode',
       'Seg6Departure', 'Seg6Arrival', 'Seg6Date', 'Seg6Carrier',
       'Seg6BookingCode', 'From', 'IsPublishedForNeg', 'IsFromInternet',
       'IsFromVista', 'TerminalID', 'InternetOffice'],
      dtype='object')

## Plan A

**Plan action**
1. Prepare dataframes: 
    - Bookings:
        - Rename columns
        - Extract date with regex
        - Convert columns to more suitable dtypes to save memory
        - Create index column
    - Searches:
        - Select *Origin*, *Destination*, *Seg1Date*
        - Rename columns
        - Convert columns to more suitable dtypes to save memory
        - Create index column
2. Join dataframes
    - Inner join between *modified Searches* and *Bookings* by *origin*, *destination* and *date*
        - Add *booked* column
        - Drop *origin*, *destination*, *date*
    - Left join between initial *Searches* and previous join by *index*
        - Fill NaNs with False
        - Drop *index*
3. Save output

### Step 1: Prepare dataframe

#### Bookings dataframe

In [8]:
mod_bookings = (
    bookings
    .rename(
        columns={"dep_port": "origin", "arr_port": "destination", "brd_time": "date"}
    )
    .assign(
        origin=bookings['dep_port'].str.strip(),
        destination=bookings['arr_port'].str.strip(),
        date=bookings['brd_time'].str.extract(r"^(\d{4}-\d{2}-\d{2})", expand=False))
    .astype(
        {"origin": "category", "destination": "category", "date": "datetime64"},
    )
    .reset_index()
)

In [9]:
mod_bookings.head()

,index,origin,destination,date
0,0,ZRH,LHR,2013-03-07
1,1,SAL,CLT,2013-04-12
2,2,SAL,CLT,2013-07-15
3,3,AKL,SVO,2013-04-24
4,4,AKL,SVO,2013-05-14


#### Searches dataframe

In [10]:
mod_searches = (
    searches
    .filter(['Origin', 'Destination', 'Seg1Date'])
    .rename(
        columns={'Origin': 'origin', 'Destination': 'destination', 'Seg1Date': 'date'}
    )
    .assign(
        origin=searches['Origin'].str.strip(),
        destination=searches['Destination'].str.strip(),
    )
    .astype(
        {"origin": "category", "destination": "category", "date": "datetime64"},
    )
    
)

In [11]:
mod_searches.head()

,origin,destination,date
0,TXL,AUH,2013-01-26
1,ATH,MIL,2013-01-04
2,ICT,SFO,2013-08-02
3,RNB,ARN,2013-01-02
4,OSL,MAD,2013-03-22


In [12]:
mod_searches.dtypes

origin               category
destination          category
date           datetime64[ns]
dtype: object

### Step 2: Join dataframes

#### Inner join between modified searches and bookings

In [13]:
join = (
    mod_searches
    .merge(
        mod_bookings, 
        how='inner', 
        on=['origin', 'destination', 'date'],
    )
    .drop_duplicates(subset='index', keep='first')
    .assign(booked=True)
    .drop(columns=['origin', 'destination', 'date'])
)


#### Left join between initial searches and previous join

In [14]:
ext_searches = (
    searches
    .reset_index()
    .merge(
        join,
        on=['index'],
        how='left',
    )
    .fillna({'booked': False})
    .rename(columns={"booked": "Booked"})
    .drop(columns='index')
)


In [15]:
ext_searches.shape

(438240, 46)

In [16]:
print(f"Convertion rate: {100*ext_searches['Booked'].mean():.2f}%")

Convertion rate: 2.99%


### Step 3: Save output

In [17]:
(
    ext_searches
    .to_csv(path_ext_searches, sep="^", index=False)
)

## Plan B

**Plan action**
1. Prepare Bookings dataframes: 
    - Extract date with regex from brd_time column
    - Add booked column to True
2. Left join between initial *Searches* and modified *Bookings*
    - Fill NaNs with False
    - Drop "dep_port*, *arr_port*, *brd_time*, *index*
3. Save output

### Step 1: Prepare Bookings dataframes

In [18]:
mod_bookings2 = (
    bookings
    .apply(lambda col: col.str.strip() if col.dtypes == "object" else col)
    .assign(
        date=bookings["brd_time"].str.extract(r"^(\d{4}-\d{2}-\d{2})", expand=False),
        booked=True
    )
)

In [19]:
mod_bookings2.head()

,dep_port,arr_port,brd_time,date,booked
0,ZRH,LHR,2013-03-07 08:50:00,2013-03-07,True
1,SAL,CLT,2013-04-12 13:04:00,2013-04-12,True
2,SAL,CLT,2013-07-15 07:00:00,2013-07-15,True
3,AKL,SVO,2013-04-24 23:59:00,2013-04-24,True
4,AKL,SVO,2013-05-14 20:15:00,2013-05-14,True


### Step 2: Left join between initial *Searches* and modified *Bookings*

In [20]:
ext_searches2 = (
    searches
    .reset_index()
    .merge(
        mod_bookings2,
        left_on=["Origin", "Destination", "Seg1Date"],
        right_on=["dep_port", "arr_port", "date"],
        how='left',
    )
    .drop_duplicates(subset="index", keep="first")
    .fillna({'booked': False})
    .drop(columns = ["dep_port", "arr_port", "brd_time", "date", "index"])
    .rename(columns={"booked": "Booked"})
)

In [21]:
ext_searches2.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,Booked
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,NaN,NaN,NaN,1ASIWS,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,FRA,False
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,NaN,NaN,NaN,1ASIWS,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,KIV,False
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,NaN,NaN,NaN,1ASIWS,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,NYC,False
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,NaN,NaN,NaN,1ASI,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,STO,False
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,NaN,NaN,NaN,1ASIWS,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,OSL,True


In [22]:
ext_searches2.shape

(438240, 46)

In [23]:
print(f"Convertion rate: {100*ext_searches2['Booked'].mean():.2f}%")

Convertion rate: 5.13%


### Step 3: Save output

In [24]:
(
    ext_searches2
    .to_csv(path_ext_searches2, sep="^", index=False)
)